In [ ]:
import pandas as pd
import io

In [ ]:
from google.colab import files 
uploaded = files.upload()

Saving movie_review.csv to movie_review.csv


In [ ]:
df = pd.read_csv(io.BytesIO(uploaded['movie_review.csv'])) 
df.head()

,fold_id,cv_tag,html_id,sent_id,text,tag
0,0,cv000,29590,0,films adapted from comic books have had plenty...,pos
1,0,cv000,29590,1,"for starters , it was created by alan moore ( ...",pos
2,0,cv000,29590,2,to say moore and campbell thoroughly researche...,pos
3,0,cv000,29590,3,"the book ( or "" graphic novel , "" if you will ...",pos
4,0,cv000,29590,4,"in other words , don't dismiss this film becau...",pos


In [ ]:
import pandas as pd
import numpy as np
import math
import matplotlib.pyplot as plt
import re
import glob
import random
import seaborn as sns
import string

##Preprocessing the Dataset

A particular review text starts with <s and ends with /s> in the clean_text column


In [ ]:
df['clean_text'] = df['text'].str.replace('[{}]'.format(string.punctuation), '')
df['clean_text'] = df['clean_text'].str.lower()
df['clean_text'] = '<s ' + df['clean_text']
df['clean_text'] = df['clean_text'] + ' /s>'

df.head()

,fold_id,cv_tag,html_id,sent_id,text,tag,clean_text
0,0,cv000,29590,0,films adapted from comic books have had plenty...,pos,<s films adapted from comic books have had ple...
1,0,cv000,29590,1,"for starters , it was created by alan moore ( ...",pos,<s for starters it was created by alan moore ...
2,0,cv000,29590,2,to say moore and campbell thoroughly researche...,pos,<s to say moore and campbell thoroughly resear...
3,0,cv000,29590,3,"the book ( or "" graphic novel , "" if you will ...",pos,<s the book or graphic novel if you will ...
4,0,cv000,29590,4,"in other words , don't dismiss this film becau...",pos,<s in other words dont dismiss this film beca...


In [ ]:
import nltk
from nltk import word_tokenize
from nltk.util import ngrams
from collections import Counter
nltk.download('punkt')

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


True

## Trigram Model Implementation


In [ ]:
df=df.head(1000)

In [ ]:
from collections import defaultdict
from nltk import bigrams, trigrams
from collections import Counter
model = defaultdict(lambda: defaultdict(lambda: 0))

#to store count of a token that follows a trigram
for sentence in df['clean_text']:
  #Split the sentence into trigrams in order to count the frequencies of the trigrams
  for w1, w2, w3 in trigrams(sentence.split(), pad_right=True, pad_left=True):
    model[(w1, w2)][w3] += 1
 

In [ ]:
model

defaultdict(<function __main__.<lambda>>,
            {(None,
              None): defaultdict(<function __main__.<lambda>.<locals>.<lambda>>, {'<s': 1000}),
             (None,
              '<s'): defaultdict(<function __main__.<lambda>.<locals>.<lambda>>, {'/s>': 3,
                          '1996s': 1,
                          '2': 1,
                          'a': 10,
                          'abberline': 1,
                          'above': 1,
                          'absolutely': 1,
                          'according': 1,
                          'after': 10,
                          'again': 1,
                          'aka': 1,
                          'alda': 1,
                          'aldas': 1,
                          'all': 3,
                          'ally': 1,
                          'almost': 3,
                          'along': 1,
                          'also': 5,
                          'although': 7,
                          'amidala': 1,
  

In [ ]:
#prints the number of times "plot points" is followed by "and"
print(model[('plot','points')]['and'])

2


In [ ]:
#to store the probability of a token following a trigram
for w1_w2 in model:
    total_count = float(sum(model[w1_w2].values()))
    for w3 in model[w1_w2]:
        # To calculate the probability of the model
        model[w1_w2][w3] /= total_count

In [ ]:
model

defaultdict(<function __main__.<lambda>>,
            {(None,
              None): defaultdict(<function __main__.<lambda>.<locals>.<lambda>>, {'<s': 1.0}),
             (None,
              '<s'): defaultdict(<function __main__.<lambda>.<locals>.<lambda>>, {'/s>': 0.002999999999999998,
                          '1996s': 0.0009999999999999994,
                          '2': 0.0009999999999999994,
                          'a': 0.009999999999999993,
                          'abberline': 0.0009999999999999994,
                          'above': 0.0009999999999999994,
                          'absolutely': 0.0009999999999999994,
                          'according': 0.0009999999999999994,
                          'after': 0.009999999999999993,
                          'again': 0.0009999999999999994,
                          'aka': 0.0009999999999999994,
                          'alda': 0.0009999999999999994,
                          'aldas': 0.0009999999999999994,
                

In [ ]:
#probability that the token "great" follows the sequence "and a"
model[('and','a')]['great']

0.0625

## Predicing the sentence

In [ ]:
import random

In [ ]:
#choosing a set of starting words
text = ["in", "the"]
sentence_finished = False

In [ ]:
#predicting the sentence given the starting words
while not sentence_finished:
  # select a random probability threshold  
  r = random.random()
  accumulator = .0

  for word in model[tuple(text[-2:])].keys():
      accumulator += model[tuple(text[-2:])][word]
      # select words that are above the probability threshold
      if accumulator >= r:
          text.append(word)
          break

  if text[-2:] == [None, None]:
      sentence_finished = True

print (' '.join([t for t in text if t]))

in the process this creates for a good job of going into ground zero and attempting to talk a man who has used the sudden popularity of the specialist and pulp fiction which avary cowrote with quentin tarantino /s>
